In [2]:
#!python3.6
import subprocess as sps
import numpy as np
import os
import time
import threading
import math
import cv2
from PIL import ImageGrab
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
import pygame
import random
import pyautogui
import keyboard
import csv
import discord
import asyncio
import pyDes
import sys
import wx
import shutil
#os.chdir("../")
pygame.init()
clock = pygame.time.Clock()

In [3]:
class cmdmaker:
    
    shot = False
    cmdmaker_threading_OFF_flag = False
    
    def __init__(self,washa,washaName):
        self.cv = washa
        self.washaName = washaName
        self.tasklist = []
        self.speeknow = 0
        self.innerclock = pygame.time.Clock()
        self.cmd = 'seikasay.exe この音声が流れることはないよ'
        self.coment = 'デフォルトです'
        self.frame = 30
        self.mabataki = 4
        self.kutipaku = 4
        self.vanish = False
        self.faceconfig = 0
        self.Red = 0
        self.Green = 0
        self.Blue = 0
        self.text = ""
        self.changeText = False
        self.textTime = 0
        self.RunNow = False
        self.JimakuNow = False
        self.TachieNow = False
        self.stop_Talk = True
        self.posenum = 0
        self.chouseiConfig = []
        self.vanishtimer = 0
        self.View_face = False
        if os.path.isdir('./voiceConfig'):
            if os.path.exists("voiceConfig/" + self.washaName + ".csv"):
                self.chouseiConfig = getcsv("voiceConfig/" + self.washaName + ".csv")
                self.chouseiConfig.pop(0)

        
    def makecmd(self):
        chousei_is_success = False
        for chousei in self.chouseiConfig:
            if str(self.tasklist[0]) == chousei[0]:
                if len(chousei) == 4:
                    self.cmd = 'seikasay.exe -cid ' + str(self.cv) + ' -speed ' + chousei[1] + ' -pitch ' + chousei[2] + ' -intonation ' + chousei[3] + ' \"' + str(self.tasklist[2]) + '\"'
                    chousei_is_success = True
                    break
                elif len(chousei) == 7:
                    self.cmd = 'seikasay.exe -cid ' + str(self.cv) + ' -speed ' + chousei[1] + ' -pitch ' + chousei[2] + ' -intonation ' + chousei[3] + ' -happiness ' + chousei[4] + ' -hatred ' + chousei[5] + ' -sadness ' + chousei[6] + ' \"' + str(self.tasklist[2]) + '\"'
                    chousei_is_success = True
                    break                    
        if not chousei_is_success:
            self.cmd = 'seikasay.exe -cid ' + str(self.cv) + ' \"' + str(self.tasklist[2]) + '\"'
        
        

    
    def getTaskList(self,tlist):
        self.tasklist.append(tlist)
        
    
    def runVoiceRoid(self):
        print(self.tasklist)
        if len(self.tasklist) == 3:
            self.makecmd()
            self.textTime = 0
            self.posenum = int(self.tasklist[1])
            self.faceconfig = int(self.tasklist[0])
            self.text = self.tasklist[2]
            self.View_face = True
            self.changeText = True
            self.speeknow +=1
            sps.call(self.cmd,shell=True,timeout = 15)
            self.speeknow -=1
            self.textTime = 1
            self.tasklist = []
            self.change_look = []
    

            
    def set_vanish_time(self,vnish_time):
        if vanish_time < 31:
            self.vanishtimer = 900 - (vanish_time * 30)
            if self.vanishtimer == 0:
                self.vanishtimer = 1
        


    def DrawMyChar(self):
        cv2.namedWindow(self.washaName)
        body = []
        eye = []
        pose = []
        im = Image.new('RGB', (1980, 1080), (0, 255, 0))
        im = np.asarray(im)
        im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
        haikei = im
        if os.path.isdir("./data"):
            datafiles = os.listdir("data/" + self.washaName + "/")
            faceMax = len(datafiles)
        
            for datafile in datafiles:
                bdls = []
                eyels = []
                if os.path.isdir("data/" + self.washaName + "/" + datafile + "/body"):
                    files = os.listdir("data/" + self.washaName + "/" + datafile + "/body/")
                    if len(files) > 0:
                        for file in files:
                            if ".png" in file:
                                mp = cv2.imread("data/" + self.washaName + "/" + datafile + "/body/" + file,-1)
                                mp = cv2.resize(mp,(int(mp.shape[1]*750/mp.shape[0]),750))
                                bdls.append(mp)
                        body.append(bdls)
                    else:
                        body.append([])
                    
                if os.path.isdir("data/" + self.washaName + "/" + datafile + "/eye"):
                    files = os.listdir("data/" + self.washaName + "/" + datafile + "/eye/")
                    if len(files) > 0:
                        for file in files:
                            if ".png" in file:
                                ep = cv2.imread("data/" + self.washaName + "/" + datafile + "/eye/" + file,-1)
                                ep = cv2.resize(ep,(int(ep.shape[1]*750/ep.shape[0]),750))
                                eyels.append(ep)
                        eye.append(eyels)
                    else:
                        eye.append([])

            if os.path.isdir("./pose"):
                pols = []
                if os.path.isdir("pose/" + self.washaName + "/"):
                    files = os.listdir("data/" + self.washaName + "/")
                    if len(files) > 0:
                        for file in files:
                            if ".png" in file:
                                mp = cv2.imread("data/" + self.washaName + "/" + file,-1)
                                mp = cv2.resize(mp,(int(mp.shape[1]*750/mp.shape[0]),750))
                                pols.append(mp)
                        pose.append(pols)
                    else:
                        pose.append([])      
                    
                
            animeclock = pygame.time.Clock()
            timer = 0
            eyetime = 0
            eyestart = False
            eyescene = 0
            speekNow = True
            faceA = 0 #表情
            posenumA = 0 #ポーズの番号
            bodynumA = 0 #表情の番号
            eyenumA = 0 #目の番号
            faceB = 0 #表情
            posenumB = 0 #ポーズの番号
            bodynumB = 0 #表情の番号
            eyenumB = 0 #目の番号
        #初期化
            drawmatrix = haikei.copy()
            if len(eye[faceA]) > 0:
                maskE = eye[faceA][0][:,:,3].copy()
                ret,maskE = cv2.threshold(maskE, 1, 1, cv2.THRESH_BINARY)
                maskE = cv2.cvtColor(maskE, cv2.COLOR_GRAY2BGR)
            if len(body[faceA]) > 0:
                maskB = body[faceA][0][:,:,3].copy()
                ret,maskB = cv2.threshold(maskB, 1, 1, cv2.THRESH_BINARY)
                maskB = cv2.cvtColor(maskB, cv2.COLOR_GRAY2BGR)

            if len(pose) > 0:
                maskP = pose[0][:,:,3].copy()
                ret,maskP = cv2.threshold(maskP, 1, 1, cv2.THRESH_BINARY)
                maskP = cv2.cvtColor(maskP, cv2.COLOR_GRAY2BGR)
        
            while True:
                needchange = False
                        
                if speekNow and self.speeknow == 0:#話終わり
                    self.vanishtimer = 1
                    bodynumA = 0
                    speekNow = False
                    needchange = True
            
                if not speekNow and self.speeknow > 0: #話はじめ
                    self.vanishtimer = 0
                    speekNow = True
                    if self.faceconfig < faceMax:
                        faceA = self.faceconfig
                    else:
                        faceA = 0
                    if posenumA < len(pose):
                        posenumA = self.posenum
                    else:
                        posenumA = 0
                    needchange = True
                    eyenumA = 0
                    bodynumA = 0
               

                    
        
                if speekNow:
                    self.vanishtimer = 0
                    if len(body[faceA]) > 0:        #bodyがあれば
                        if timer % self.kutipaku == 0:
                            if len(body[faceA]) > 2:
                                bodynumA += random.randrange(len(body[faceA])-2)+ 1
                                if bodynumA >= len(body[faceA]):
                                    bodynumA -= len(body[faceA])-1
                            elif len(body[faceA]) == 2:
                                bodynumA = 1
                            else:
                                bodynumA = 0

        
                elif self.vanishtimer < 150:

                    if self.vanish and self.vanishtimer > 0 and self.vanishtimer < 900:
                        self.vanishtimer += 1
        
                
                if self.View_face:
                    if self.faceconfig < faceMax:
                        faceA = self.faceconfig
                    else:
                        faceA = 0
                    if posenumA < len(pose):
                        posenumA = self.posenum
                    else:
                        posenumA = 0
                    self.vanishtimer = 1
                    needchange = True
                    self.View_face = False            
            
                
                
        
        
                if len(eye[faceA]) > 0:   #eyeがあればここは、speeknowと非同期
                    eyetime += 1
                    if eyetime > self.mabataki * self.frame and random.randrange(3) == 0:
                        eyetime = 0
                        if len(eye[faceA]) > 1:
                            eyescene = 0
                            eyestart = True
                    if eyestart:
                        eyenumA = len(eye[faceA]) - 1 - abs(len(eye[faceA]) - 1 - eyescene)
                        eyescene += 1
                        if int(eyenumA) == -1:
                            eyenumA = 0
                            eyescene = 0
                            eyestart = False
                        
                    if len(eye[faceA]) - 1 < eyenumA:
                        eyenumA = 0
                        eyescene = 0
                        eyestart = False

                        

    
                if not eyenumA == eyenumB:
                    needchange = True
                    eyenumB = eyenumA
                    
                if not bodynumA == bodynumB:
                    needchange = True
                    bodynumB = bodynumA
                    
                if not posenumA == posenumB:
                    needchange = True
                    posenumB = posenumA
                    
                if not faceA == faceB:
                    needchange = True
                    faceB = faceA

                
                if needchange and self.vanishtimer < 900:
                    drawmatrix = haikei.copy()

                    
                    
                    if len(eye[faceA]) > 0:
                        maskE = eye[faceA][eyenumA][:,:,3].copy()
                        ret,maskE = cv2.threshold(maskE, 1, 1, cv2.THRESH_BINARY)
                        maskE = cv2.cvtColor(maskE, cv2.COLOR_GRAY2BGR)
                        
                    if len(body[faceA]) > 0:
                        maskB = body[faceA][bodynumA][:,:,3].copy()
                        ret,maskB = cv2.threshold(maskB, 1, 1, cv2.THRESH_BINARY)
                        maskB = cv2.cvtColor(maskB, cv2.COLOR_GRAY2BGR)

                    if len(pose) > 0:
                        maskP = pose[posenumA][:,:,3].copy()
                        ret,maskP = cv2.threshold(maskP, 1, 1, cv2.THRESH_BINARY)
                        maskP = cv2.cvtColor(maskP, cv2.COLOR_GRAY2BGR)
                
                    if len(body[faceA]) > 0:
                        drawmatrix = cv2.resize(drawmatrix,(body[faceA][bodynumA].shape[1],body[faceA][bodynumA].shape[0]))
                        drawmatrix *= 1 - maskB
                        drawmatrix += body[faceA][bodynumA][:,:,:3] * maskB
                        
                        if len(eye[faceA]) > 0:
                            drawmatrix = cv2.resize(drawmatrix,(eye[faceA][eyenumA].shape[1],eye[faceA][eyenumA].shape[0]))
                            drawmatrix *= 1 - maskE
                            drawmatrix += eye[faceA][eyenumA][:,:,:3] * maskE
                            
                        if len(pose) > 0:
                            drawmatrix = cv2.resize(drawmatrix,(pose[posenumA].shape[1],pose[posenumA].shape[0]))
                            drawmatrix *= 1 - maskP
                            drawmatrix += pose[posenumA][:,:,:3] * maskP

                    else:
                        drawmatrix = cv2.resize(drawmatrix,(450,750))

                    

                    
            
                if self.vanishtimer == 900:
                    drawmatrix = haikei.copy()
                    drawmatrix = cv2.resize(drawmatrix,(450,750))
                    self.vanishtimer = 901
            
            
                cv2.imshow(self.washaName,drawmatrix)
                cv2.waitKey(1)  
                
                
                timer += 1
                if timer == 60:
                    timer = 0
                if cmdmaker.cmdmaker_threading_OFF_flag:
                    break
                animeclock.tick(self.frame)
            
            
    def startChar(self):
        if not self.TachieNow:
            if os.path.isdir("./data"):
                if os.path.isdir("data/" + self.washaName):
                    Tachie = threading.Thread(target = self.DrawMyChar)
                    Tachie.start()
                    self.TachieNow = True
        
        
        
    def WriteJimaku(self):
        cv2.namedWindow(self.washaName + "Jimaku")
        im = Image.new('RGB', (1980, 100), (0, 255, 0))
        textbox = im.copy()
        textbox = np.asarray(textbox)
        textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
        Jimakuclock = pygame.time.Clock()
        while True:
            if self.changeText:
                self.changeText = False
                textbox = im.copy()
                draw = ImageDraw.Draw(textbox)
                font_size = 48
                while True:
                    draw.font = ImageFont.truetype("C:\Windows\Fonts\HGRPP1.ttc",font_size)
                    img_size = np.array(textbox.size)
                    txt_size = np.array(draw.font.getsize(self.text))
                    if img_size[0] > txt_size[0]:
                        break
                    font_size -= 2
        
                pos = (img_size - txt_size) / 2
                draw.text(pos, self.text, fill=(self.Red, self.Green, self.Blue))
                textbox = np.asarray(textbox)
                textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
                
                
            if self.textTime == 300:
                self.textTime = 0
                textbox = im.copy()
                textbox = np.asarray(textbox)
                textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
                
            if self.textTime > 0:
                self.textTime += 1
            cv2.imshow(self.washaName + "Jimaku",textbox)
            cv2.waitKey(1)
            if cmdmaker.cmdmaker_threading_OFF_flag:
                break
            Jimakuclock.tick(self.frame)
            
    def startJimaku(self):
        t1 = threading.Thread(target = self.WriteJimaku)
        t1.start()
        self.JimakuNow = True

In [4]:
class Jimaku():
    def __init__(self):
        self.text = ""
        self.Red = 0
        self.Green = 0
        self.Blue = 0
        self.changeText = False
        self.textTime = 0
        
    def StartJimaku(self):
        cv2.namedWindow("Jimaku")
        im = Image.new('RGB', (1600, 200), (0, 255, 0))
        textbox = im.copy()
        textbox = np.asarray(textbox)
        textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
        Jimakuclock = pygame.time.Clock()

        while True:
            if self.changeText:
                self.textTime = 0
                self.changeText = False
                textbox = im.copy()
                draw = ImageDraw.Draw(textbox)
                font_size = 54
                while True:
                    draw.font = ImageFont.truetype("C:\Windows\Fonts\HGRPP1.ttc",font_size)
                    img_size = np.array(textbox.size)
                    txt_size = np.array(draw.font.getsize(self.text))
                    if img_size[0] > txt_size[0]:
                        break
                    font_size -= 2
        
                pos = (img_size - txt_size) / 2
                draw.text(pos, self.text, fill=(self.Red, self.Green, self.Blue))
                textbox = np.asarray(textbox)
                textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
                
                
            if self.textTime == 300:
                self.textTime = 0
                textbox = im.copy()
                textbox = np.asarray(textbox)
                textbox = cv2.cvtColor(textbox,cv2.COLOR_BGR2RGB)
                
            if self.textTime > 0:
                self.textTime += 1
            cv2.imshow("Jimaku",textbox)
            cv2.waitKey(1)

            Jimakuclock.tick(30)
            
    def startJimaku(self):
        t1 = threading.Thread(target = self.StartJimaku)
        t1.start()

In [5]:
class voiceRoidManager():
    
    voiceRoidManager_threading_OFF_flag = False
    
    def __init__(self):
        self.VoiceRoidList = []
        self.tasklist = []
        self.make_move = False
        self.tweet = False
        self.tText = ""
        self.waitTime = 0
        self.recX = 0
        self.recY = 0
        self.ButtonXY = []



    
    
    def appendVoiceRoid(self,cvID,washaName,R,G,B):
        self.VoiceRoidList.append(cmdmaker(cvID,washaName))
        self.VoiceRoidList[-1].Red = int(R)
        self.VoiceRoidList[-1].Green = int(G)
        self.VoiceRoidList[-1].Blue = int(B)
        
    def changeRGB2(self,washaName,R,G,B):
        for vList in self.VoiceRoidList:
            if vList.washaName == washaName:
                vList.Red = R
                vList.Green = G
                vList.Blue = B
                
    def changeRGB2(self,num,R,G,B):
        self.VoiceRoidList[num].Red = R
        self.VoiceRoidList[num].Green = G
        self.VoiceRoidList[num].Blue = B

    def vanish_change2(self,washaName):
        for vList in self.VoiceRoidList:
            if vList.washaName == washaName:
                if vList.vanish:
                    vList.vanish = False
                else:
                    vList.vanish = True
                    
    def vanish_change(self,num):
        if len(self.VoiceRoidList) > num:
            if self.VoiceRoidList[num].vanish:
                self.VoiceRoidList[num].vanish = False
            else:
                self.VoiceRoidList[num].vanish = True
                
                
                
    

    def sortingList(self):
        while len(self.tasklist) > 0:
            if str(self.tasklist[0][0]) == "talk":
                Jtext = str(self.tasklist[0][4][:])
                while True:
                    if "<rws>" in Jtext and "<rwe>" in Jtext:
                        Jtext = Jtext.split("<rws>",maxsplit = 1)[0] + Jtext.split("<rwe>",maxsplit = 1)[1]
                    else:
                        Ji.text = Jtext.replace("<rws>","").replace("<rwe>","").replace("<rts>","").replace("<rte>","")
                        break
                        
                for vList in self.VoiceRoidList:
                    if self.tasklist[0][1] == vList.washaName:
                        Ji.Red = vList.Red
                        Ji.Green = vList.Green
                        Ji.Blue = vList.Blue
                        break
                Ji.changeText = True
                while True:
                    if "<rts>" in self.tasklist[0][4] and "<rte>" in self.tasklist[0][4]:
                        self.tasklist[0][4] = self.tasklist[0][4].split("<rts>",maxsplit = 1)[0] + self.tasklist[0][4].split("<rte>",maxsplit = 1)[1]
                    else:
                        self.tasklist[0][4] = self.tasklist[0][4].replace("<rws>","").replace("<rwe>","").replace("<rts>","").replace("<rte>","")
                        break                
                
                
                if "、" in self.tasklist[0][4]:
                    splist = self.tasklist[0][4].split("、")
                    for hoge in splist:
                        
                        #while True:
                            #if "<rts>" in hoge and "<rte>" in hoge:
                                #hoge = hoge.split("<rts>",maxsplit = 1)[0] + Jtext.split("<rte>",maxsplit = 1)[1]
                            #else:
                                #hoge = hoge.replace("<rws>","").replace("<rwe>","").replace("<rts>","").replace("<rte>","")
                                #break
                                
                        for vList in self.VoiceRoidList:
                            if self.tasklist[0][1] == vList.washaName:
                                vList.tasklist = [self.tasklist[0][2],self.tasklist[0][3],str(hoge)]
                                vList.runVoiceRoid()
                                time.sleep(0.1)
                                break
                else:
                    for vList in self.VoiceRoidList:
                        if self.tasklist[0][1] == vList.washaName:
                            
                            #while True:
                                #if "<rts>" in self.tasklist[0][4] and "<rte>" in self.tasklist[0][4]:
                                    #self.tasklist[0][4] = self.tasklist[0][4].split("<rts>",maxsplit = 1)[0] + Jtext.split("<rte>",maxsplit = 1)[1]
                                #else:
                                    #self.tasklist[0][4] = self.tasklist[0][4].replace("<rws>","").replace("<rwe>","").replace("<rts>","").replace("<rte>","")
                                    #break
                                    
                            vList.tasklist = self.tasklist[0][2:5]
                            vList.runVoiceRoid()
                            break
                Ji.textTime = 1

                        
            if str(self.tasklist[0][0]) == "wait":
                try:
                    wai = int(self.tasklist[0][1])
                    time.sleep(wai)
                except:
                    pass
                
            if str(self.tasklist[0][0]) == "tweet":
                self.tText = str(self.tasklist[0][1])
                self.tweet = True

                
            if str(self.tasklist[0][0]) == "Vanish":
                self.vanish_change2(str(self.tasklist[0][1]))

            if str(self.tasklist[0][0]) == "changeWait":
                self.waitTime = float(self.tasklist[0][1])
            
            i = 0
            for bt in self.ButtonXY:
                if str(self.tasklist[0][0]) == "Button" + str(i):
                    pyautogui.click(self.ButtonXY[i][0],self.ButtonXY[i][1], 1, 0, 'left')
                i += 1
                    
            if str(self.tasklist[0][0]) == "Record":
                pyautogui.click(self.recX,self.recY, 1, 0, 'left')
                
            if str(self.tasklist[0][0]) == "changeVanishTime":
                washa = str(self.tasklist[0][1])
                timer = int(self.tasklist[0][2])
                for vList in self.VoiceRoidList:
                    if washa == str(vList.washaName):
                        vList.tasklist.set_vanish_time(timer)
                        break
                        
                        
                
                
            self.tasklist.pop(0)
            time.sleep(self.waitTime)

            


                    
    def startAllVoiceRoid(self):
        if len(self.VoiceRoidList) > 0:
            for vList in self.VoiceRoidList:
                if not vList.RunNow:
                    vList.startJimaku()
                    vList.startChar()
                    vList.RunNow = True

In [6]:
class discord_manager:
    
    def __init__(self):
        self.token = ''
        self.get_command = False
        self.client = discord.Client()

  
    def set_discord(self,token):
        self.token = token

                        
    def start_discord(self): 
        self.client.run(self.token)
        
    def start_threading_discord(self):
        disc = threading.Thread(target = self.start_discord)
        disc.start()


Ji = Jimaku()
dis = discord_manager()
manager = voiceRoidManager()

        
        
@dis.client.event
async def on_message(message):
    if message.content.startswith("!m-"):
        if dis.client.user != message.author:
            msg = message.content
            msg_list = msg.split("-")[1:]
            if msg_list[0] == "system":
                print("get system call")
                if msg_list[1] == "setupV":
                    manager.appendVoiceRoid(int(msg_list[2]),str(msg_list[3]),int(msg_list[4]),int(msg_list[5]),int(msg_list[6]))
                    manager.startAllVoiceRoid()
                    print("start VoiceRoid")
                if msg_list[1] == "Vanish":
                    manager.vanish_change2(msg_list[2])
                if msg_list[1] == "getScript":
                    if not manager.make_move:
                        dis.get_command = True
                        print("get Script Start")
                if msg_list[1] == "endScript":
                    dis.get_command = False
                    manager.make_move = True
                if msg_list[1] == "clear":
                    manager.tasklist = []
            else:
                if dis.get_command:
                    manager.tasklist.append(msg_list)




In [ ]:
def getcsv(path):
    if os.path.isfile(path):
        with open(path, 'r') as f:
            reader = list(csv.reader(f))
        return reader
    return []

In [ ]:
if __name__ == '__main__':

    if os.path.exists("token.csv"):
        tokenList = getcsv("token.csv")
        for tlss in tokenList:
            if str(tlss[0]) == "discord":
                dis.token = tlss[1]
                print("get dicord token")
            if str(tlss[0]) == "Twitter":
                twitterToken = tlss[1]
                
    dis.start_threading_discord()
    print("discord start")
    Ji.startJimaku()
    time.sleep(10)
    manager.recX,manager.recY = pyautogui.position()
    print("recX = " + str(manager.recX))
    print("recY = " + str(manager.recY))
    
    
    for num in range(6):
        time.sleep(3)
        x,y = pyautogui.position()
        manager.ButtonXY.append([x,y])
        print("Button"+str(num) +"_X:" +str(x))
        print("Button"+str(num) +"_Y:" +str(y))
    

    
    
    
    
                
    while True:
        while not manager.make_move:
            time.sleep(1)
            if os.path.isdir("./script"):
                files = os.listdir("script/")
                for file in files:
                    if ".csv" in file:
                        manager.tasklist = getcsv("script/" + file)
                        print(manager.tasklist)
                        shutil.move("script/" + file,"complete/")
                        manager.make_move = True
                        break
        
        time.sleep(5)
        manager.sortingList()
        time.sleep(20)

    #ファイルを移動させる処理
        while manager.make_move:
            videofile = os.listdir("video/")
            if len(videofile) > 0:
                for vl in videofile:
                #if manager.tweet:
                    #twitterに送る処理
                    shutil.move("video/" + vl,"complete/")
                manager.tweet = False
                manager.make_move = False
                break
            time.sleep(3)
            


get dicord token
discord start
recX = 1531
recY = 587
Button0_X:146
Button0_Y:586
Button1_X:153
Button1_Y:612
Button2_X:504
Button2_Y:669
Button3_X:768
Button3_Y:843
get system call
start VoiceRoid
Button4_X:737
Button4_Y:713
Button5_X:758
Button5_Y:840
get system call
start VoiceRoid
[['changeWait', '0.2'], ['talk', 'Akane', '1', '0', '噂の天才プログラマー、あかねちゃんやで！'], ['talk', 'Akane', '5', '0', '今日はイージームービーメイカーっちゅうもん作っててな'], ['talk', 'Akane', '5', '0', '今はテスト運用中やで！'], ['talk', 'Aoi', '3', '0', 'これどういうシステムなの？'], ['talk', 'Akane', '5', '0', 'これはな、簡単お手軽、編集せいへんでも動画がかってに出来上がるっちゅうツールやな'], ['talk', 'Aoi', '1', '0', 'へー！すごいね！'], ['talk', 'Aoi', '0', '0', '<rts>立ち絵<rte><rws>たちえ<rwe>とか字幕を自動で入れられるのはわかったけど、他に何ができるのかな？'], ['talk', 'Akane', '2', '0', 'たぶん、音楽とか流せると思うんやけど、どうやろ'], ['talk', 'Akane', '2', '0', 'とりあえずやってみるしかないでな、というわけで'], ['talk', 'Akane', '1', '0', 'ミュージックスタート！'], ['Button0'], ['wait', '3'], ['talk', 'Akane', '5', '0', '後は嘘字幕やな'], ['talk', 'Akane', '5', '0', '<rws>こんな風にしゃべりながら、別の字幕を出す機能やな<rwe><

In [ ]:
manager = voiceRoidManager()

In [6]:
manager.appendVoiceRoid(2000,"Akane",255,100,100)
manager.startAllVoiceRoid()

In [8]:
manager.appendVoiceRoid(2001,"Aoi",100,100,255)
manager.startAllVoiceRoid()

In [17]:
msg_list1 = ["talk","Akane",1,0,"うちはあかねちゃんやで"]
msg_list2 = ["talk","Akane",2,0,"というわけで、いろいろテストしていくで"]
msg_list3 = ["talk","Aoi","3","0","実際にちゃんと動くかの調査だね"]
msg_list4 = ["talk","Akane",4,0,"ときよとまれ！"]
msg_list5 = ["wait",5]
msg_list6 = ["talk","Akane",6,0,"多分止まってたはずや"]
msg_list7 = ["talk","Aoi",2,0,"ちゃんと機能しているみたいだね"]
msg_list8 = ["talk","Akane",8,0,"これがちゃんと動いてるなら、なかなかええんとちゃう？"]


In [18]:
manager.tasklist.append(msg_list1)
manager.tasklist.append(msg_list2)
manager.tasklist.append(msg_list3)
manager.tasklist.append(msg_list4)
manager.tasklist.append(msg_list5)
manager.tasklist.append(msg_list6)
manager.tasklist.append(msg_list7)
manager.tasklist.append(msg_list8)
manager.sortingList()

[['talk', 'Akane', 1, 0, 'うちはあかねちゃんやで'], ['talk', 'Akane', 2, 0, 'というわけで、いろいろテストしていくで'], ['talk', 'Aoi', '3', '0', '実際にちゃんと動くかの調査だね'], ['talk', 'Akane', 4, 0, 'ときよとまれ！'], ['wait', 5], ['talk', 'Akane', 6, 0, '多分止まってたはずや'], ['talk', 'Aoi', 2, 0, 'ちゃんと機能しているみたいだね'], ['talk', 'Akane', 8, 0, 'これがちゃんと動いてるなら、なかなかええんとちゃう？']]
[1, 0, 'うちはあかねちゃんやで']
[['talk', 'Akane', 2, 0, 'というわけで、いろいろテストしていくで'], ['talk', 'Aoi', '3', '0', '実際にちゃんと動くかの調査だね'], ['talk', 'Akane', 4, 0, 'ときよとまれ！'], ['wait', 5], ['talk', 'Akane', 6, 0, '多分止まってたはずや'], ['talk', 'Aoi', 2, 0, 'ちゃんと機能しているみたいだね'], ['talk', 'Akane', 8, 0, 'これがちゃんと動いてるなら、なかなかええんとちゃう？']]
[2, 0, 'というわけで、いろいろテストしていくで']
[['talk', 'Aoi', '3', '0', '実際にちゃんと動くかの調査だね'], ['talk', 'Akane', 4, 0, 'ときよとまれ！'], ['wait', 5], ['talk', 'Akane', 6, 0, '多分止まってたはずや'], ['talk', 'Aoi', 2, 0, 'ちゃんと機能しているみたいだね'], ['talk', 'Akane', 8, 0, 'これがちゃんと動いてるなら、なかなかええんとちゃう？']]
['3', '0', '実際にちゃんと動くかの調査だね']
[['talk', 'Akane', 4, 0, 'ときよとまれ！'], ['wait', 5], ['talk', 'Akane', 6, 0, '多分止まってたはずや'

In [1]:
for num in range(5):
    print(num)

0
1
2
3
4


In [7]:
a = 0.001
print(float(a))

0.001
